In [ ]:
!apt-get update
!apt-get install -y \
    xfce4 xfce4-goodies \
    x11vnc xvfb novnc websockify python3-pyinotify \
    firefox

!pip install pyngrok


In [ ]:
import os
os.environ["NGROK_AUTH_TOKEN"] = "YOUR_NGROK_TOKEN"
!ngrok config add-authtoken <add ngrok authtoken>

In [ ]:
import subprocess, threading, time, os
from pyngrok import ngrok

# Set a VNC password, stored in /tmp/vncpass
vnc_password = "notebook"
!x11vnc -storepasswd {vnc_password} /tmp/vncpass

def run_command(cmd_list):
    subprocess.run(cmd_list, check=True)

# 1. Start Xvfb (virtual display) on :1
thread_xvfb = threading.Thread(
    target=run_command,
    args=(["Xvfb", ":1", "-screen", "0", "1366x768x16", "-ac", "-pn", "-noreset"],),
    daemon=True
)
thread_xvfb.start()
time.sleep(2)

# 2. Start x11vnc to serve that display
thread_x11vnc = threading.Thread(
    target=run_command,
    args=(["x11vnc", "-display", ":1", "-rfbauth", "/tmp/vncpass", "-forever", "-shared", "-rfbport", "5901"],),
    daemon=True
)
thread_x11vnc.start()
time.sleep(2)

# 3. Start the XFCE desktop environment on :1
#    This gives you a more familiar desktop (panels, menus, etc.).
os.environ["DISPLAY"] = ":1"
thread_xfce = threading.Thread(
    target=run_command,
    args=(["startxfce4"],),
    daemon=True
)
thread_xfce.start()
time.sleep(5)  # Give XFCE a few seconds to initialize

# 4. Automatically launch Firefox in the XFCE session
subprocess.Popen(["firefox"], env={"DISPLAY":":1"})

# 5. Start noVNC/websockify on port 6901
#    This serves an HTML5 VNC client at http://localhost:6901/vnc.html
thread_noVNC = threading.Thread(
    target=run_command,
    args=(["websockify", "--web", "/usr/share/novnc/", "6901", "localhost:5901"],),
    daemon=True
)
thread_noVNC.start()
time.sleep(3)

# 6. Create a public URL with ngrok on port 6901 using HTTP
public_url = ngrok.connect(6901, "http")
print("noVNC is running. Please open the following URL in a new browser tab:")
print(public_url.public_url + "/vnc.html")

print("\nWhen prompted for a password, enter:", vnc_password)

